In [1]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
import pytz

In [2]:
import os
import numpy as np
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/master project/data/AAPL' #เข้า drive

Mounted at /content/drive


In [3]:
api_key = 'J4AM5YKF88UA41WO'

In [4]:
def _get_data(symbols, time_from, time_to, api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

In [5]:
def get_dataset(time_from="20190101T0000",
                time_to="20250301T0000",
                MAX_API_CALLS_PER_DAY = 99999, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 300 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%600==0:
            time.sleep(65)

        data=_get_data('AAPL',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract AAPL specific relevance (we didn't use it in video)

    # Only take tickers with AAPL in headline
    df=df[df.title.str.contains('AAPL|apple',case=False)]
    # Only take when # of tickers = 1
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    #df.set_index('time_published',inplace=True)
    # Sort by time published
    #df.sort_index(inplace=True)
    return df


In [6]:
df = get_dataset()

In [7]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.tz_localize('UTC')

In [8]:
df['time_published']=df['time_published'].dt.tz_convert('US/Eastern')

In [9]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.strftime('%Y-%m-%d')

In [10]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,Buffett's On The Sidelines - Should You Follow...,https://www.benzinga.com/markets/equities/25/0...,2025-02-28,[MarketBeat],Warren Buffett and Berkshire Hathaway ( NYSE: ...,https://www.marketbeat.com/logos/articles/med_...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Energy & Transportation', 'relevan...",0.082801,Neutral,"[{'ticker': 'AXP', 'relevance_score': '0.10400..."
1,Warren Buffett's Subtle Message To Trump: 'Spe...,https://www.benzinga.com/news/25/02/44049686/w...,2025-02-28,[Kaili Killpack],"Warren Buffett, the legendary investor and CEO...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.374552,Bullish,"[{'ticker': 'BAC', 'relevance_score': '0.11894..."
2,Competitor Analysis: Evaluating Apple And Comp...,https://www.benzinga.com/insights/news/25/02/4...,2025-02-28,[Benzinga Insights],In today's rapidly changing and highly competi...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.287114,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.5157..."
7,Great News for Apple Stock Investors,https://www.fool.com/investing/2025/02/28/grea...,2025-02-28,"[Parkev Tatevosian, CFA]",Apple ( NASDAQ: AAPL ) stock investors are cau...,https://g.foolcdn.com/editorial/images/809364/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.383535,Bearish,"[{'ticker': 'AAPL', 'relevance_score': '0.8760..."
10,Tim Cook's Apple Is Developing New Technology ...,https://www.benzinga.com/25/02/44036079/apples...,2025-02-28,[Rounak Jain],Apple Inc. AAPL is working on introducing a ne...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Real Estate & Construction', 'rele...",0.187926,Somewhat-Bullish,"[{'ticker': 'APG', 'relevance_score': '0.10158..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZ...",https://www.zacks.com/stock/news/1878481/googl...,2022-03-07,[Zacks Investment Research],Alphabet's (GOOGL) Google peps up audio stream...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.114362,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.4334..."
473,Here's How Wedbush Views Apple Ahead Of Its Ma...,https://www.benzinga.com/news/22/03/26016559/h...,2022-03-07,[],Wedbush analyst Dan Ives rated Apple Inc ( NA...,https://cdn.benzinga.com/files/imagecache/stor...,Benzinga,News,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.074536,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.6077..."
480,"Stocks Tumble, Oil Surges, Inflation, Amex And...",https://www.thestreet.com/markets/-5-things-yo...,2022-03-07,[Martin Baccardax],Stock futures tumble as Russia-Ukraine war int...,https://www.thestreet.com/.image/ar_16:9%2Cc_f...,The Street,n/a,www.thestreet.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.094578,Neutral,"[{'ticker': 'PYPL', 'relevance_score': '0.0977..."
481,Apple Co-Founder Says Many Cryptocurrencies Ar...,https://www.benzinga.com/markets/cryptocurrenc...,2022-03-07,[Madhukumar Warrier],Apple Inc. ( NASDAQ: AAPL ) co-founder Steve...,https://cdn2.benzinga.com/files/imagecache/og_...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.021708,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.5053..."


In [11]:
News_AAPL = df[["time_published", "title", "summary","overall_sentiment_score"]]

In [12]:
News_AAPL

,time_published,title,summary,overall_sentiment_score
0,2025-02-28,Buffett's On The Sidelines - Should You Follow...,Warren Buffett and Berkshire Hathaway ( NYSE: ...,0.082801
1,2025-02-28,Warren Buffett's Subtle Message To Trump: 'Spe...,"Warren Buffett, the legendary investor and CEO...",0.374552
2,2025-02-28,Competitor Analysis: Evaluating Apple And Comp...,In today's rapidly changing and highly competi...,0.287114
7,2025-02-28,Great News for Apple Stock Investors,Apple ( NASDAQ: AAPL ) stock investors are cau...,-0.383535
10,2025-02-28,Tim Cook's Apple Is Developing New Technology ...,Apple Inc. AAPL is working on introducing a ne...,0.187926
...,...,...,...,...
471,2022-03-07,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZ...",Alphabet's (GOOGL) Google peps up audio stream...,0.114362
473,2022-03-07,Here's How Wedbush Views Apple Ahead Of Its Ma...,Wedbush analyst Dan Ives rated Apple Inc ( NA...,-0.074536
480,2022-03-07,"Stocks Tumble, Oil Surges, Inflation, Amex And...",Stock futures tumble as Russia-Ukraine war int...,-0.094578
481,2022-03-07,Apple Co-Founder Says Many Cryptocurrencies Ar...,Apple Inc. ( NASDAQ: AAPL ) co-founder Steve...,-0.021708


In [13]:
!pip install nltk

In [14]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [15]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
sia = SentimentIntensityAnalyzer()
# Analyze sentiment for each summary

x = str(News_AAPL['summary']).encode("utf-8")
News_AAPL['sentiment_score'] = News_AAPL['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])

<ipython-input-16-e9a7a8b9c31e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  News_AAPL['sentiment_score'] = News_AAPL['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [17]:
News_AAPL

,time_published,title,summary,overall_sentiment_score,sentiment_score
0,2025-02-28,Buffett's On The Sidelines - Should You Follow...,Warren Buffett and Berkshire Hathaway ( NYSE: ...,0.082801,0.0000
1,2025-02-28,Warren Buffett's Subtle Message To Trump: 'Spe...,"Warren Buffett, the legendary investor and CEO...",0.374552,0.0000
2,2025-02-28,Competitor Analysis: Evaluating Apple And Comp...,In today's rapidly changing and highly competi...,0.287114,0.3597
7,2025-02-28,Great News for Apple Stock Investors,Apple ( NASDAQ: AAPL ) stock investors are cau...,-0.383535,0.5423
10,2025-02-28,Tim Cook's Apple Is Developing New Technology ...,Apple Inc. AAPL is working on introducing a ne...,0.187926,0.4019
...,...,...,...,...,...
471,2022-03-07,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZ...",Alphabet's (GOOGL) Google peps up audio stream...,0.114362,0.1779
473,2022-03-07,Here's How Wedbush Views Apple Ahead Of Its Ma...,Wedbush analyst Dan Ives rated Apple Inc ( NA...,-0.074536,0.0000
480,2022-03-07,"Stocks Tumble, Oil Surges, Inflation, Amex And...",Stock futures tumble as Russia-Ukraine war int...,-0.094578,-0.8020
481,2022-03-07,Apple Co-Founder Says Many Cryptocurrencies Ar...,Apple Inc. ( NASDAQ: AAPL ) co-founder Steve...,-0.021708,0.0000


In [18]:
News_AAPL_Nozero=News_AAPL.loc[News_AAPL['sentiment_score'] != 0]
News_AAPL_Nozero

,time_published,title,summary,overall_sentiment_score,sentiment_score
2,2025-02-28,Competitor Analysis: Evaluating Apple And Comp...,In today's rapidly changing and highly competi...,0.287114,0.3597
7,2025-02-28,Great News for Apple Stock Investors,Apple ( NASDAQ: AAPL ) stock investors are cau...,-0.383535,0.5423
10,2025-02-28,Tim Cook's Apple Is Developing New Technology ...,Apple Inc. AAPL is working on introducing a ne...,0.187926,0.4019
11,2025-02-28,Tesla Stock vs. Apple Stock: Billionaires Buy ...,Tesla ( NASDAQ: TSLA ) and Apple ( NASDAQ: AAP...,0.113703,0.2609
12,2025-02-27,Cathie Wood Sells Off Spotify Stock As YouTube...,"On Thursday, Cathie Wood's Ark Invest made a s...",0.162259,0.4588
...,...,...,...,...,...
455,2022-03-07,Tech Stocks Roundup: What Will Apple Announce ...,Check out the latest news and performance numb...,-0.070630,0.6124
460,2022-03-07,Gridiron Streaming Fight: Could Apple Beat Ama...,Two of the largest technology companies in the...,0.025978,-0.6369
471,2022-03-07,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZ...",Alphabet's (GOOGL) Google peps up audio stream...,0.114362,0.1779
480,2022-03-07,"Stocks Tumble, Oil Surges, Inflation, Amex And...",Stock futures tumble as Russia-Ukraine war int...,-0.094578,-0.8020


In [19]:
daily_avg_sentiment_AAPL = News_AAPL.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_AAPL

,time_published,sentiment_score
0,2022-03-01,-0.079000
1,2022-03-07,-0.085090
2,2022-03-08,0.202393
3,2022-03-09,0.283187
4,2022-03-10,0.149844
...,...,...
1073,2025-02-24,0.225800
1074,2025-02-25,0.120367
1075,2025-02-26,0.202040
1076,2025-02-27,0.606100


In [20]:
daily_avg_sentiment_Nozero_AAPL = News_AAPL_Nozero.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_Nozero_AAPL

,time_published,sentiment_score
0,2022-03-01,-0.079000
1,2022-03-07,-0.170180
2,2022-03-08,0.435923
3,2022-03-09,0.377583
4,2022-03-10,0.168575
...,...,...
1058,2025-02-24,0.301067
1059,2025-02-25,0.154757
1060,2025-02-26,0.202040
1061,2025-02-27,0.606100


In [21]:
daily_avg_sentiment_AAPL.to_csv(os.path.join(path,'daily_avg_sentiment_AAPL.csv'))
News_AAPL.to_csv(os.path.join(path,'News_AAPL.csv'))

In [22]:
daily_avg_sentiment_Nozero_AAPL.to_csv(os.path.join(path,'daily_avg_sentiment_Nozero_AAPL.csv'))
News_AAPL_Nozero.to_csv(os.path.join(path,'News_AAPL_Nozero.csv'))